## Speech Recognition

A survey of statistical and deep learning models

## Automatic Speech Recognition Pipeline

<img src='assets/speech/asr-pipeline.jpg'/>


Source: https://www.techrepublic.com/article/how-we-learned-to-talk-to-computers/

## Speech Feature Extraction

<img src='assets/speech/Spectrogram-19thC.png'/>

A spectrogram for "nineteen century" - power vs. frequency

Common method: Mel-frequency cepstral coefficients (MFCC)

## Statistical Speech Recognition

$$W^* = \underset{W}{\operatorname{argmax}}P(W|X)$$

- word sequence: $W$
- most likely word sequence: $W^*$
- acoustic input feature vector (e.g. MFCC): $X$

## Statistical Speech Recognition

After Bayes' Theorem:

$$W^* = \underset{W}{\operatorname{argmax}}p(X|W)P(W)$$

- acoustic model: $p(X|W)$
- language model (e.g. N-gram): $P(W)$

## Statistical Acoustic Model: $p(X|W)$

<img src='assets/speech/acoustic-statistical.png' width='50%'/>

Credits: https://www.inf.ed.ac.uk/teaching/courses/asr/2016-17/asr03-hmmgmm-handout.pdf

## Hidden Markov Model: $p(S_i|S_{i-1})$, Gaussian Mixture Model: $p(X|S_i)$

<img src='assets/speech/acoustic-hmm-gmm.png' width='50%'/>

Credits: https://www.inf.ed.ac.uk/teaching/courses/asr/2016-17/asr03-hmmgmm-handout.pdf

## Gaussian Mixture Model

Mixture distribution: combine multiple probabability distributions to make an improved model

$$P(x) = \sum_iP(c=i)P(x \mid c=i)$$

$i^{th}$ Gaussian component: $P(x \mid c=i)$

Applications
- Clustering
- Classification

Nice intro:
https://yulearning.blogspot.sg/2014/11/einsteins-most-famous-equation-is-emc2.html

## Workshop: GMM gender detector
---

Credits: https://github.com/abhijeet3922/PyGender-Voice

<img src='assets/speech/workshop1_pygender.png' style='float:right'/>

1. Download data from [here](
https://www.dropbox.com/s/hcku4t7alrhacqv/pygender.zip?dl=0)

2. Extract the .zip file to a folder of your choice. Note down the path as you will need to enter it in the workshop code.

In [ ]:
# !pip3 install python_speech_features

import os
from os.path import basename, join
import numpy as np

from python_speech_features import mfcc
from scipy.io.wavfile import read
from sklearn import preprocessing
from sklearn.mixture import GaussianMixture

TRAIN_PATH = 'C:\\mldds\\pygender\\train_data\\youtube\\' # modify to your actual path

In [ ]:
def get_MFCC(audio_file, scale=True):
    """Computes the Mel-frequency ceptrum coefficients for an audio file
    Args:
        audio_file: input audio file
        scale: whether to scale the features between 0-1
    Returns:
        the MFCC features
    """    
    sample_rate, audio = read(audio_file)
    features = mfcc(audio, sample_rate, winlen=0.025, winstep=0.01,
                    numcep=13, appendEnergy=False)
    if scale:
        features = preprocessing.scale(features) # scale to (0, 1)
    return features

In [ ]:
# Playback a sample file
from IPython import display

sample_file = join(TRAIN_PATH, 'male', 'male1.wav')
sample_rate, audio = read(sample_file)
display.Audio(data=audio, rate=sample_rate)

In [ ]:
# Plot the MFCC
import matplotlib.pyplot as plt

mfcc_vector = get_MFCC(sample_file, scale=False)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,4))
cax = ax.matshow(np.transpose(mfcc_vector), interpolation='nearest',
                 aspect='auto', cmap='coolwarm', origin='lower')
fig.colorbar(cax)
plt.title("Spectrogram of {}".format(sample_file))
plt.show()

In [ ]:
def train_GMM(data_path, n_components=8, covariance_type='diag'):
    """Trains a Gaussian mixture model
    Args:
        data_path: data folder path
        n_components: number of Gaussian components
        covariance_type: type of covariance matrix
    Returns:
        the GMM model
    """    
    files = [join(data_path, f) for f in os.listdir(data_path) if f.endswith('.wav')]
    features = np.asarray(());

    for f in files:
        mfcc_vector = get_MFCC(f)

        if features.size:
            features = np.vstack((features, mfcc_vector))
        else:
            features = mfcc_vector

    gmm = GaussianMixture(n_components=n_components,
                          covariance_type=covariance_type,
                          max_iter=200, n_init=3)
    gmm.fit(features)
    
    # print some metrics applicable to GMMs
    print('BIC: ', gmm.bic(features), ', AIC: ', gmm.aic(features))
    return gmm

In [ ]:
models = dict()
%time models['male'] = train_GMM(join(TRAIN_PATH, 'male'), n_components=8, \
                                 covariance_type='diag')

# ==================================================================
# Exercise:
# Add code below to train the female model, using the above as an example








# ==================================================================
# Optional Exercises:
# a. Try different values of n_component (e.g. 2, 16)
# b. Try different values of covariance_type (e.g. full)
#
# See http://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html
# on how to interpret the BIC and AIC metrics for selecting models









In [ ]:
def test_GMM(models, test_data_path):
    """Trains a Gaussian mixture model
    Args:
        models: {name: gmm} dictionary of models
        test_data_path: test data folder path
    Returns:
        the predictions
    """    
    files = [os.path.join(test_data_path,f) for f in os.listdir(test_data_path)
             if f.endswith(".wav")]
    
    predictions = []
    for f in files:
        features = get_MFCC(f)
        keys = []
        log_likelihood = np.zeros(len(models))

        for i, (key, gmm) in enumerate(models.items()):
            scores = np.array(gmm.score(features))
            keys.append(key)
            log_likelihood[i] = scores.sum()

        # find the model with the maximum score
        winner = np.argmax(log_likelihood)
        # print('prediction:', keys[winner], "\tscores:", log_likelihood[winner])
        predictions.append(keys[winner])
    return predictions

In [ ]:
# ==================================================================
# Exercise:
# 1. Complete the code below to test the GMM models using test_GMM().
#    Be sure to run against both male and female models.
# 2. Plot the confusion matrix

from sklearn.metrics import confusion_matrix

TEST_PATH = 'C:\\mldds\\pygender\\test_data\\AudioSet' # modify to your actual path










## Shortcomings of Statistical Approaches

Lots of hand-tuning

Inefficient for approximating non-linear data: combination covariance matrices get very large / complicated

Solution: deep learning

## Deep Belief Nets

[Paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/38131.pdf)

- Replaces the Gaussian Mixture Model in an ASR system 
- Generative pre-training
 - Speeds up training with less overfitting
 - Train a hidden layer (using Restricted Boltmann Machines or Gaussian RBM)
 - Use the weights as inputs to train next layer
 - Stack up into a forward-only "Deep Belief Net" (DBN)
 - Add softmax to create the DBM-DNN

Python: https://pypi.org/project/nolearn/

![DBN-DNN](assets/speech/dbn-dnn.png)

![DBN-DNN performance](assets/speech/dbn-dnn-compare.png)

## Word Error Rate

$$WER = \frac{S + D + I}{N}$$

- substitution word count: $S$
- deletion word count: $D$
- insertion word count: $I$
- correct word count: $C$
- number of reference words: $N = S + D + C$

## Word Accuracy
$$WAcc = 1 - WER = \frac{(N - S - D) - I}{N} = \frac{C - I}{N} $$

## Recurrent Neural Networks

[Paper](http://proceedings.mlr.press/v32/graves14.pdf)

End-to-End Speech Recognition
- Transcription is hard. Skip it and train model that converts speech directly to text
- Bidirectional LSTM to learn long sequences
- Correctionist Temporal Classification (CTC) to align audio with text 

## LSTM

- Learns long sequences (like speech)
- Forget gate learns what to forget

![lstm](assets/speech/lstm.png)

## Birectional RNN

- Forward ($\overrightarrow{h}$) and backward ($\overleftarrow{h}$) hidden sequences
- Learns context in both directions (like words in speech)
- Bidirectional LSTM: LSTM hidden units 

![BRNN](assets/speech/brnn.png)

## Correctionist Temporal Classification

[Paper](https://www.cs.toronto.edu/~graves/icml_2006.pdf)

- To train speech to text, we need to know how audio "lines up" with transcripts
- Input sequences: $X$ (such as audio)
- Output sequences: $Y$ (such as transcripts)

## Correctionist Temporal Classification

Problem: $X$ and $Y$ can vary in length, in different ways
![CTC](assets/speech/naive_alignment.svg)

Solution: introduce the blank token ($\epsilon$)
![CTC](assets/speech/ctc_alignment_steps.svg)

Process: train an RNN to estimate probabilities of each character per time step

## Correctionist Temporal Classification

![CTC](assets/speech/full_collapse_from_audio.svg)

[Visual Guide and Explanation](https://distill.pub/2017/ctc/)

## Walkthrough: CTC
---
Credits: https://github.com/igormq/ctc_tensorflow_example

In [18]:
# https://github.com/lspvic/jupyter_tensorboard
# !pip3 install jupyter-tensorboard

import os
import sys
import numpy as np
from os.path import join
from python_speech_features import mfcc
from scipy.io.wavfile import read
from sklearn import preprocessing
import tensorflow as tf
import time

In [19]:
# Constants
SPACE_TOKEN = '<space>'
SPACE_INDEX = 0
FIRST_INDEX = ord('a') - 1  # 0 is reserved to space

# Configuration
num_features = 13
num_units = 50 # Number of units in the LSTM cell

# Number of classes: 'a' to 'z' +  space + blank label = 28 characters
num_classes = ord('z') - ord('a') + 1 + 1 + 1

# Hyper-parameters
num_epochs = 200
num_hidden = 50
num_layers = 1
batch_size = 1
initial_learning_rate = 1e-2
momentum = 0.9

num_examples = 1
num_batches_per_epoch = int(num_examples/batch_size)

In [20]:
def process_audio(audio_file):
    """Extracts the MFCC features from an audio file.
    Args:
        audio_file: file containing the input audio
    Returns:
        A tuple with (features, sequence length)
    """
    sample_rate, audio = read(audio_file)
    features = mfcc(audio, sample_rate)
    features = preprocessing.scale(features) # scale to (0, 1)

    features = np.asarray(features[np.newaxis, :])
    features_seq_len = [features.shape[1]]
    return features, features_seq_len

# Process audio
TRAIN_INPUT = join('files', 'CTC', 'LDC93S1.wav')
train_input, train_seq_len = process_audio(TRAIN_INPUT)

VAL_INPUT = join('files', 'CTC', '61-70968-0002.wav')
val_input, val_seq_len = process_audio(VAL_INPUT)

In [21]:
def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representation of sequences.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape

def process_targets(text_file):
    """Extracts the CTC tokens from an input text file.
    Args:
        text_file: file containing the input text
    Returns:
        A nested tuple with (original text, (indices, CTC tokens, shape))
    """
    with open(text_file, 'r') as f:
        line = f.readlines()[-1] # take the last line

        # Get only the words between [a-z], replace period for none
        original = ' '.join(line.strip().lower().split(' ')).replace('.', '')
        targets = original.replace(' ', '  ')
        targets = targets.split(' ')

    # Add blank label
    targets = np.hstack([SPACE_TOKEN if x == '' else list(x) for x in targets])

    # Transform char into index
    targets = np.asarray([SPACE_INDEX if x == SPACE_TOKEN else ord(x) - FIRST_INDEX
                          for x in targets])

    # Creat sparse representation to feed into the graph
    return original, sparse_tuple_from([targets])

# Process transcription
TRAIN_TARGET = join('files', 'CTC', 'LDC93S1.txt')
train_original, train_targets = process_targets(TRAIN_TARGET)
print("Original text:", train_original)
print("Targets:", train_targets[1])

VAL_TARGET = join('files', 'CTC', '61-70968-0002.txt')
val_original, val_targets = process_targets(VAL_TARGET)
print("Original text:", val_original)
print("Targets:", val_targets[1])

Original text: your dark suit in greasy wash water all year
Targets: [25 15 21 18  0  4  1 18 11  0 19 21  9 20  0  9 14  0  7 18  5  1 19 25
  0 23  1 19  8  0 23  1 20  5 18  0  1 12 12  0 25  5  1 18]
Original text: fortune and a happy life
Targets: [ 6 15 18 20 21 14  5  0  1 14  4  0  1  0  8  1 16 16 25  0 12  9  6  5]


In [22]:
graph = tf.Graph()
with graph.as_default():
    # e.g: log filter bank or MFCC features
    # Has size [batch_size, max_stepsize, num_features], but the
    # batch_size and max_stepsize can vary along each step
    inputs = tf.placeholder(tf.float32, [None, None, num_features])

    # Here we use sparse_placeholder that will generate a
    # SparseTensor required by ctc_loss op.
    targets = tf.sparse_placeholder(tf.int32)

    # 1d array of size [batch_size]
    seq_len = tf.placeholder(tf.int32, [None])

    # Defining the cell
    # Can be:
    #   tf.nn.rnn_cell.RNNCell
    #   tf.nn.rnn_cell.GRUCell 
    cells = []
    for _ in range(num_layers):
        cell = tf.contrib.rnn.LSTMCell(num_units)
        cells.append(cell)
    stack = tf.contrib.rnn.MultiRNNCell(cells)

    # The second output is the last state and we will no use that
    outputs, _ = tf.nn.dynamic_rnn(stack, inputs, seq_len, dtype=tf.float32)

    shape = tf.shape(inputs)
    batch_s, max_timesteps = shape[0], shape[1]

    # Reshaping to apply the same weights over the timesteps
    outputs = tf.reshape(outputs, [-1, num_hidden])

    # Truncated normal with mean 0 and stdev=0.1
    W = tf.Variable(tf.truncated_normal([num_hidden,
                                         num_classes],
                                        stddev=0.1))
    # Zero initialization
    b = tf.Variable(tf.constant(0., shape=[num_classes]))

    # Doing the affine projection
    logits = tf.matmul(outputs, W) + b

    # Reshaping back to the original shape
    logits = tf.reshape(logits, [batch_s, -1, num_classes])

    # Time major
    logits = tf.transpose(logits, (1, 0, 2))

    loss = tf.nn.ctc_loss(targets, logits, seq_len)
    cost = tf.reduce_mean(loss)

    optimizer = tf.train.MomentumOptimizer(initial_learning_rate,
                                           0.9).minimize(cost)

    # Option 2: tf.nn.ctc_beam_search_decoder
    # (it's slower but you'll get better results)
    decoded, log_prob = tf.nn.ctc_greedy_decoder(logits, seq_len)

    # Inaccuracy: label error rate
    ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32),
                                          targets))
    
    # TensorBoard summary operation
    tf.summary.scalar('ctc_cost', cost)
    tf.summary.scalar('label_error_rate', ler)
    summary_op = tf.summary.merge_all()

In [23]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()

    # TensorBoard
    train_summary_writer = tf.summary.FileWriter('./logs/ctc/train', session.graph)
    val_summary_writer = tf.summary.FileWriter('./logs/ctc/val', session.graph)

    for curr_epoch in range(num_epochs):
        train_cost = train_ler = 0
        start = time.time()

        for batch in range(num_batches_per_epoch):

            feed = {inputs: train_input,
                    targets: train_targets,
                    seq_len: train_seq_len}

            batch_cost, _, summary = session.run([cost, optimizer, summary_op], feed)
            train_summary_writer.add_summary(summary, curr_epoch * num_batches_per_epoch + batch)
            
            train_cost += batch_cost*batch_size
            train_ler += session.run(ler, feed_dict=feed)*batch_size
            
        train_cost /= num_examples
        train_ler /= num_examples

        val_feed = {inputs: val_input,
                    targets: val_targets,
                    seq_len: val_seq_len}

        val_cost, val_ler, val_summary = session.run([cost, ler, summary_op], feed_dict=val_feed)
        val_summary_writer.add_summary(val_summary, curr_epoch * num_batches_per_epoch * batch_size)
        
        log = "Epoch {}/{}, train_cost = {:.3f}, train_ler = {:.3f}, val_cost = {:.3f}, val_ler = {:.3f}, time = {:.3f}"
        print(log.format(curr_epoch+1, num_epochs, train_cost, train_ler,
                         val_cost, val_ler, time.time() - start))
    # Decoding
    d = session.run(decoded[0], feed_dict=val_feed)
    str_decoded = ''.join([chr(x) for x in np.asarray(d[1]) + FIRST_INDEX])

    # Replacing blank label to none
    str_decoded = str_decoded.replace(chr(ord('z') + 1), '')

    # Replacing space label to space
    str_decoded = str_decoded.replace(chr(ord('a') - 1), ' ')

    print('Original:\n%s' % val_original)
    print('Decoded:\n%s' % str_decoded)

Epoch 1/200, train_cost = 774.809, train_ler = 1.000, val_cost = 1115.457, val_ler = 1.000, time = 0.225
Epoch 2/200, train_cost = 497.494, train_ler = 1.000, val_cost = 96.116, val_ler = 1.000, time = 0.096
Epoch 3/200, train_cost = 218.598, train_ler = 1.000, val_cost = 129.305, val_ler = 1.000, time = 0.094
Epoch 4/200, train_cost = 253.486, train_ler = 1.000, val_cost = 127.988, val_ler = 1.000, time = 0.120
Epoch 5/200, train_cost = 144.481, train_ler = 0.886, val_cost = 452.867, val_ler = 0.958, time = 0.102
Epoch 6/200, train_cost = 156.072, train_ler = 0.886, val_cost = 387.177, val_ler = 0.958, time = 0.105
Epoch 7/200, train_cost = 131.799, train_ler = 1.000, val_cost = 130.367, val_ler = 1.000, time = 0.112
Epoch 8/200, train_cost = 143.932, train_ler = 0.909, val_cost = 245.622, val_ler = 0.833, time = 0.121
Epoch 9/200, train_cost = 130.099, train_ler = 0.864, val_cost = 296.762, val_ler = 0.708, time = 0.122
Epoch 10/200, train_cost = 132.673, train_ler = 0.818, val_cost 

Epoch 81/200, train_cost = 9.869, train_ler = 0.000, val_cost = 475.664, val_ler = 1.875, time = 0.123
Epoch 82/200, train_cost = 9.394, train_ler = 0.000, val_cost = 481.864, val_ler = 1.833, time = 0.107
Epoch 83/200, train_cost = 8.923, train_ler = 0.000, val_cost = 486.831, val_ler = 1.917, time = 0.119
Epoch 84/200, train_cost = 8.481, train_ler = 0.000, val_cost = 490.295, val_ler = 2.042, time = 0.116
Epoch 85/200, train_cost = 8.066, train_ler = 0.000, val_cost = 492.891, val_ler = 2.042, time = 0.144
Epoch 86/200, train_cost = 7.671, train_ler = 0.000, val_cost = 495.005, val_ler = 2.042, time = 0.133
Epoch 87/200, train_cost = 7.291, train_ler = 0.000, val_cost = 496.810, val_ler = 1.958, time = 0.130
Epoch 88/200, train_cost = 6.935, train_ler = 0.000, val_cost = 498.655, val_ler = 1.875, time = 0.115
Epoch 89/200, train_cost = 6.611, train_ler = 0.000, val_cost = 500.398, val_ler = 1.875, time = 0.122
Epoch 90/200, train_cost = 6.308, train_ler = 0.000, val_cost = 501.689, 

Epoch 160/200, train_cost = 0.905, train_ler = 0.000, val_cost = 568.878, val_ler = 2.333, time = 0.140
Epoch 161/200, train_cost = 0.892, train_ler = 0.000, val_cost = 569.613, val_ler = 2.333, time = 0.123
Epoch 162/200, train_cost = 0.879, train_ler = 0.000, val_cost = 570.345, val_ler = 2.333, time = 0.133
Epoch 163/200, train_cost = 0.867, train_ler = 0.000, val_cost = 571.077, val_ler = 2.333, time = 0.150
Epoch 164/200, train_cost = 0.855, train_ler = 0.000, val_cost = 571.810, val_ler = 2.292, time = 0.154
Epoch 165/200, train_cost = 0.843, train_ler = 0.000, val_cost = 572.544, val_ler = 2.292, time = 0.137
Epoch 166/200, train_cost = 0.832, train_ler = 0.000, val_cost = 573.274, val_ler = 2.292, time = 0.147
Epoch 167/200, train_cost = 0.820, train_ler = 0.000, val_cost = 573.996, val_ler = 2.292, time = 0.161
Epoch 168/200, train_cost = 0.809, train_ler = 0.000, val_cost = 574.706, val_ler = 2.292, time = 0.144
Epoch 169/200, train_cost = 0.799, train_ler = 0.000, val_cost =

## Deep Speech

Deep Speech 1: https://arxiv.org/abs/1412.5567

Deep Speech 2: https://arxiv.org/abs/1512.02595

## Workshop: Deep Speech